Formatted station coordinate list with bash, manually removed unicode chars.

tail -n +22 $(echo TG_STAID0000*.txt) | cut -d, -f1,3,4 | awk -F, '{if($2>20100101)print}' | awk -F, '{if($3>-9999)print}' | wc -l

TODO 8/12: 

strip characters (+, :) from the station coordinates.
Divide by 10,000.
use printf with .4f?
drop duplicates. 

sed 's/[+:]//g' station_coords.csv | awk -F, '{printf "%s, %0.f, %0.f\n", $1, $2/10000, $3/10000}' | awk -F, '!seen[$2,$3]++'

merge / average the station data in those grids.


In [99]:
def load_coarsened_weather_data():
    low_res = cfgrib.open_datasets('../data/low_res_2023.grib')

    low_res_core_data = low_res[0]
    filtered_low_res = low_res[0].dropna('latitude', how="all")
    
    site_df = filtered_low_res.to_dataframe().dropna().drop(["number", "step", "surface"], axis=1).reset_index().rename(columns={"skt":"temperature"})

    #site_df['site_id'] = site_id

    site_df['year'] = site_df.time.dt.to_period('Y')
    site_df['doy'] = site_df.time.dt.strftime('%j').astype(int)

    site_df['latitude'] = site_df['latitude'].astype(int)
    site_df['longitude'] = site_df['longitude'].astype(int)


    site_df = site_df[['temperature', 'year', 'doy', 'latitude', 'longitude']]

    site_df['coordstring'] = site_df['latitude'].astype(str) + site_df['longitude'].astype(str)
    
    new_site_ids = full_observations[['rounded_lat', 'rounded_lon']].drop_duplicates().reset_index().drop('index', axis=1).reset_index().rename(columns={"index":'rounded_site_id'})

    # get rounded site IDs

    new_site_ids['coordstring'] = new_site_ids['rounded_lat'].astype(str) + new_site_ids['rounded_lon'].astype(str)


    filtered_sites = site_df[site_df['coordstring'].isin(new_site_ids['coordstring'])].rename(columns={'latitude':'rounded_lat',
                                                                                               'longitude': 'rounded_lon'})
    filtered_sites = filtered_sites.merge(new_site_ids, on=['rounded_lat', 'rounded_lon', 'coordstring']).rename(columns={'rounded_lat':'latitude',
                                                                                                   'rounded_lon': 'longitude'})

    #filtered_sites.drop('coordstring', axis=1, inplace=True)
    
    ## Daylength and Leap Year corrections

    # Correct for leap years
    leap_year_key = {60: 61, 
                     91: 92, 
                     121: 122, 
                     152: 153, 
                     182: 183, 
                     213: 214, 
                     244: 245, 
                     274: 275, 
                     305: 306, 
                     335: 336}

    corrected_leap_year_histories = filtered_sites.replace({'doy': leap_year_key})

    # Day Length Correction
    corrected_leap_year_histories['daylength'] = corrected_leap_year_histories.apply(lambda row: daylength(row['doy'], row['latitude']), axis=1)

    # rename columns for plants
    corrected_leap_year_histories.rename(columns={'rounded_site_id':'site_id'}, inplace=True)
    corrected_leap_year_histories['year'] = corrected_leap_year_histories['year'].astype(str).astype(int)
    
    return corrected_leap_year_histories

# Turns a dataframe containing predictions of flowering day to a dict? Not sure what this does. 
def ripeness_data_to_dict(ripeness_data):    
    
    mean_maturation = np.mean(ripeness_data['flowering_day'])
    
    prediction_dict = {
        "full_flowering_data": ripeness_data,
        #"species_site_flowering days": list(ripeness_data['flowering_day']),
        "mean_flowering_day": np.mean(ripeness_data['flowering_day']),
        "best_model": ripeness_data['model'][0]
    }
    
    return prediction_dict

def get_ripeness_days(model_output):
    ripeness_days = {}
    
    for k in model_output:
        ripeness_days[k] = {'mean_ripeness': model_output[k]['mean_flowering_day'],
                            'sd_ripeness':  np.std(model_output[k]['species_site_flowering days'])}
    
    return ripeness_days


def aic(obs, pred, n_param):
    return len(obs) * np.log(np.mean((obs - pred)**2)) + 2*(n_param + 1)


# Trains a model with a given set of test observations and test predictors. 
def train_ripeness(observations, predictors, test_observations, test_predictors, models=['ThermalTime']):
    # set up model comparisons
    best_aic=np.inf
    best_model = None
    best_model_name = None

    # iterate through all models
    for model_name in models:
        print("running model {m}".format(m=model_name))
        
        Model = utils.load_model(model_name)
        model = Model()
        model.fit(observations, predictors, optimizer_params='practical')
        
        # predict from test observations
        print("making predictions for model {m}".format(m=model_name))        
        preds = model.predict(test_observations, test_predictors)
        
        #print(preds)
        test_days = test_observations.doy.values
        #print(test_days)
        # this isn't valid - need to filter by site IDs
        
        # THIS IS REALLY BAD:
        test_days = test_days[0:len(preds)]
        #print(test_days)
        
        # score model
        model_aic = aic(obs = test_days,
                        pred=preds,
                        n_param = len(model.get_params()))

        if model_aic < best_aic:
            best_model = model
            best_model_name = model_name
            best_aic = model_aic

        print('model {m} got an aic of {a}'.format(m=model_name,a=model_aic))

    print('Best model: {m}'.format(m=best_model_name))
    print('Best model paramters:')
    print(best_model.get_params())
    print("Ripeness Day: {}".format(np.mean(preds)))
    
    ripeness_data = test_observations
    ripeness_data['flowering_day'] = preds
    ripeness_data['model'] = best_model_name
    
    prediction_dict = {
        "trained_model": best_model,
        "model_aic": best_aic,
        "model_name": best_model_name,
        "full_flowering_data": ripeness_data,
        "species_site_flowering days": list(ripeness_data['flowering_day']),
        "mean_flowering_day": np.mean(ripeness_data['flowering_day'])
    }
    
    return prediction_dict
    

In [178]:
import pandas as pd
import numpy as np
import os
import cfgrib
import xarray as xr


from pyPhenology import models, utils

from tqdm import trange, tqdm

from daylength import daylength

import warnings
warnings.filterwarnings('default')

In [45]:
full_observations = pd.read_csv("../data/model_training_data/all_plants_formatted.csv", index_col=0)

full_observations['rounded_lat'] = round(full_observations['latitude']).astype(int)
full_observations['rounded_lon'] = round(full_observations['lon_360']).astype(int)

In [51]:
station_coords = pd.read_csv('../data/formatted_station_coords.csv', names=["site_id", "latitude", "longitude"])

In [39]:
station_coords

,site_id,lat,lon
0,1,57,14
1,2,60,15
2,3,65,17
3,4,58,15
4,6,59,13
...,...,...,...
1401,25857,40,15
1402,25858,40,14
1403,25981,42,13
1404,25984,41,13


In [12]:
file_list = list("site" + station_coords['station'].astype(str) + ".csv")

In [13]:
file_list

['site1.csv',
 'site2.csv',
 'site3.csv',
 'site4.csv',
 'site6.csv',
 'site8.csv',
 'site10.csv',
 'site11.csv',
 'site12.csv',
 'site13.csv',
 'site14.csv',
 'site16.csv',
 'site21.csv',
 'site22.csv',
 'site24.csv',
 'site26.csv',
 'site27.csv',
 'site28.csv',
 'site29.csv',
 'site30.csv',
 'site31.csv',
 'site32.csv',
 'site33.csv',
 'site34.csv',
 'site36.csv',
 'site37.csv',
 'site40.csv',
 'site41.csv',
 'site42.csv',
 'site43.csv',
 'site44.csv',
 'site45.csv',
 'site46.csv',
 'site49.csv',
 'site50.csv',
 'site51.csv',
 'site52.csv',
 'site53.csv',
 'site55.csv',
 'site59.csv',
 'site60.csv',
 'site61.csv',
 'site62.csv',
 'site63.csv',
 'site64.csv',
 'site65.csv',
 'site66.csv',
 'site67.csv',
 'site68.csv',
 'site69.csv',
 'site70.csv',
 'site71.csv',
 'site72.csv',
 'site73.csv',
 'site74.csv',
 'site75.csv',
 'site76.csv',
 'site77.csv',
 'site78.csv',
 'site79.csv',
 'site80.csv',
 'site81.csv',
 'site82.csv',
 'site83.csv',
 'site84.csv',
 'site85.csv',
 'site86.csv',
 

In [422]:
euro_weather_data_list = []

for f in file_list:
    file_path = os.path.join("../data/formatted_euro_weather", f)
    #print(file_path)
    
    if os.path.exists(file_path):
        temp_df = pd.read_csv(file_path, names=['site_id', 'date', 'mean_temp'])
    
        euro_weather_data_list.append(temp_df)
    
full_euro_weather = pd.concat(euro_weather_data_list)

In [423]:
full_euro_weather['formatted_date'] = pd.to_datetime(full_euro_weather['date'].astype(str), format='%Y%m%d')

In [428]:
full_euro_weather['year'] = full_euro_weather.formatted_date.dt.to_period('Y').astype(str).astype(int)
full_euro_weather['doy'] = full_euro_weather.formatted_date.dt.strftime('%j').astype(int)
full_euro_weather['temperature'] = (full_euro_weather['mean_temp'] / 10)

In [429]:
full_euro_weather

,site_id,date,mean_temp,formatted_date,year,doy,temperature
0,2,20100102,-124,2010-01-02,2010,2,-12.4
1,2,20100103,-105,2010-01-03,2010,3,-10.5
2,2,20100104,-111,2010-01-04,2010,4,-11.1
3,2,20100105,-201,2010-01-05,2010,5,-20.1
4,2,20100106,-255,2010-01-06,2010,6,-25.5
...,...,...,...,...,...,...,...
4040,25991,20230224,102,2023-02-24,2023,55,10.2
4041,25991,20230225,114,2023-02-25,2023,56,11.4
4042,25991,20230226,108,2023-02-26,2023,57,10.8
4043,25991,20230227,80,2023-02-27,2023,58,8.0


In [431]:
full_euro_weather.drop(['date', 'mean_temp', 'formatted_date'], axis=1, inplace=True)

In [432]:
euro_weather_final = full_euro_weather.merge(station_coords, on="site_id")
euro_weather_final['coordstring'] = euro_weather_final['latitude'].astype(str) + euro_weather_final['longitude'].astype(str)

euro_weather_final.rename(columns={'site_id':'station'},inplace=True)


In [433]:
euro_weather_final

,station,year,doy,temperature,latitude,longitude,coordstring
0,2,2010,2,-12.4,60,15,6015
1,2,2010,3,-10.5,60,15,6015
2,2,2010,4,-11.1,60,15,6015
3,2,2010,5,-20.1,60,15,6015
4,2,2010,6,-25.5,60,15,6015
...,...,...,...,...,...,...,...
4599939,25991,2023,55,10.2,42,11,4211
4599940,25991,2023,56,11.4,42,11,4211
4599941,25991,2023,57,10.8,42,11,4211
4599942,25991,2023,58,8.0,42,11,4211


In [434]:
euro_coords = euro_weather_final['coordstring'].unique()

In [435]:
corrected_leap_year_histories = load_coarsened_weather_data()

/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/xarray/core/dataset.py:6246: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  index = self.coords.to_index([*ordered_dims])


In [436]:
corrected_leap_year_histories

,temperature,year,doy,latitude,longitude,coordstring,site_id,daylength
0,244.754715,2010,1,65,203,65203,243,3.189823
1,254.818970,2010,32,65,203,65203,243,6.205902
2,255.358810,2010,61,65,203,65203,243,9.571128
3,268.192871,2010,92,65,203,65203,243,13.154826
4,277.552795,2010,122,65,203,65203,243,16.639360
...,...,...,...,...,...,...,...,...
83425,285.150208,2023,32,-43,147,-43147,237,14.320311
83426,283.073059,2023,61,-43,147,-43147,237,13.041689
83427,281.347321,2023,92,-43,147,-43147,237,11.499389
83428,278.876526,2023,122,-43,147,-43147,237,10.084348


In [437]:
corrected_leap_year_histories[corrected_leap_year_histories['coordstring'].isin(euro_coords)]

,temperature,year,doy,latitude,longitude,coordstring,site_id,daylength
486,267.272278,2010,1,59,10,5910,359,5.975274
487,267.432251,2010,32,59,10,5910,359,7.698562
488,275.460388,2010,61,59,10,5910,359,10.127952
489,283.020996,2010,92,59,10,5910,359,12.894857
490,287.658264,2010,122,59,10,5910,359,15.504851
...,...,...,...,...,...,...,...,...
54265,285.337708,2023,32,40,16,4016,88,9.918420
54266,289.760559,2023,61,40,16,4016,88,11.063219
54267,291.226227,2023,92,40,16,4016,88,12.450401
54268,293.989807,2023,122,40,16,4016,88,13.720243


In [438]:
monthly_sites = corrected_leap_year_histories[['coordstring', 'site_id']].drop_duplicates()

In [439]:
monthly_sites

,coordstring,site_id
0,65203,243
162,65212,202
324,60209,242
486,5910,359
648,58225,533
...,...,...
82620,-38175,238
82782,-40176,482
82944,-41147,232
83106,-42147,375


In [440]:
filtered_observations = pd.read_csv('../data/model_training_data/formatted_full_observations.csv', index_col=0)

In [441]:
# Create european data from our previous "monthly sites". 
euro_monthly = euro_weather_final.merge(monthly_sites, on="coordstring")
# match with plant observations
euro_observations = filtered_observations[filtered_observations['site_id'].isin(euro_monthly['site_id'].unique())]

# format euro monthly data
euro_monthly.drop(['station', 'coordstring'], axis=1, inplace=True)

euro_monthly['year'] = euro_monthly['year'].astype(str).astype(int)

# create siteyears for matching / filtering
euro_observations['siteyear'] = euro_observations['site_id'].astype(str) + euro_observations['year'].astype(int).astype(str)
euro_monthly['siteyear'] = euro_monthly['site_id'].astype(str) + euro_monthly['year'].astype(str)

filtered_euro = euro_observations[euro_observations['siteyear'].isin(euro_monthly['siteyear'])]


incomplete_siteyears = pd.DataFrame(euro_monthly.groupby('siteyear').size()).rename(columns={0:"days"}).reset_index()
complete_siteyears = incomplete_siteyears[(incomplete_siteyears['days'] == 365) | (incomplete_siteyears['days'] == 366)]['siteyear']

filtered_euro = filtered_euro[filtered_euro['siteyear'].isin(complete_siteyears)]
euro_monthly = euro_monthly[euro_monthly['siteyear'].isin(complete_siteyears)]


/var/folders/xk/t3y_dz7j75ld1xlbfffyj5lm0000gn/T/ipykernel_67159/1386219376.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  euro_observations['siteyear'] = euro_observations['site_id'].astype(str) + euro_observations['year'].astype(int).astype(str)


In [442]:
bad_sites = []

for siteyear in euro_monthly['siteyear'].unique():
    test_doy_list = list(euro_monthly[euro_monthly['siteyear']==siteyear]['doy'])
    
    if test_doy_list != list(range(1, 366)) and test_doy_list != list(range(1, 367)):
        print(siteyear)
        
        bad_sites.append(siteyear)

492016
5032012
5262016
4882016
4252012
832016


In [443]:
filtered_euro = filtered_euro[~filtered_euro['siteyear'].isin(bad_sites)]
euro_monthly = euro_monthly[~euro_monthly['siteyear'].isin(bad_sites)]



In [444]:
filtered_euro

,doy,year,species,species_actual,phenophase,sci_name,latitude,longitude,site_id,formatted_sci_name,siteyear
1846,283.0,2015.0,Ficus,carica,516,Ficus carica,47,9,41,Ficus carica,412015
1848,179.0,2014.0,Morus,alba,516,Morus alba,47,9,41,Morus alba,412014
1849,199.0,2014.0,Prunus,cerasifera,516,Prunus cerasifera,47,9,41,Prunus cerasifera,412014
1850,197.0,2014.0,Prunus,cerasifera,516,Prunus cerasifera,47,9,41,Prunus cerasifera,412014
1851,197.0,2014.0,Prunus,cerasifera,516,Prunus cerasifera,47,9,41,Prunus cerasifera,412014
...,...,...,...,...,...,...,...,...,...,...,...
28257,164.0,2021.0,Prunus,avium,516,Prunus avium,51,18,556,Prunus avium,5562021
28258,171.0,2021.0,Prunus,avium,516,Prunus avium,44,26,557,Prunus avium,5572021
28259,171.0,2021.0,Prunus,avium,516,Prunus avium,44,26,557,Prunus avium,5572021
28260,160.0,2018.0,Prunus,avium,516,Prunus avium,51,23,558,Prunus avium,5582018


In [445]:
euro_monthly

,year,doy,temperature,latitude,longitude,site_id,siteyear
364,2011,1,-0.9,48,14,290,2902011
365,2011,2,0.4,48,14,290,2902011
366,2011,3,-1.1,48,14,290,2902011
367,2011,4,-4.9,48,14,290,2902011
368,2011,5,-7.3,48,14,290,2902011
...,...,...,...,...,...,...,...
580707,2021,361,5.3,42,13,236,2362021
580708,2021,362,5.0,42,13,236,2362021
580709,2021,363,7.5,42,13,236,2362021
580710,2021,364,10.2,42,13,236,2362021


In [446]:
species_key = filtered_euro[['species', 'formatted_sci_name']].drop_duplicates()
grouped_sites = filtered_euro.groupby(['formatted_sci_name', 'site_id']).size().reset_index().rename(columns={0:'n_obs'})
grouped_sites = grouped_sites.merge(species_key, on='formatted_sci_name')

universal_list = []

# this dict is genus: dataframe of test sites
test_dict = {}

for species in list(filtered_euro['formatted_sci_name'].unique()):
    print(species)
    species_df = grouped_sites[grouped_sites['formatted_sci_name'] == species]
    
    large_sites = list(species_df[species_df['n_obs'] >= 5]['site_id'])
    #print(species_df[species_df['n_obs'] >= 5])
    
    if len(large_sites) == 0:
        large_sites = list(species_df['site_id'])
        
    # this is printing sites with more than 5 observations.
    print(large_sites)
    
    # slice 40% of observations from filtered_observations at this species and sites
    large_species_df = filtered_euro[filtered_euro['formatted_sci_name'] == species]
    
    # 0.4 chosen somewhat arbitrarily here
    training_observations = large_species_df[large_species_df['site_id'].isin(large_sites)].sample(frac=0.4)
    #print(training_observations)
    
    # this drops all of the "training" site data
    testing_observations = large_species_df.drop(training_observations.index)
    #print(testing_observations)
    
    universal_list.append(training_observations)
    
    test_dict[species] = testing_observations
    
universal_train_df = pd.concat(universal_list)

Ficus carica
[41, 48, 49]
Morus alba
[41, 48, 134]
Prunus cerasifera
[41]
Prunus domestica
[83, 128, 150, 186, 188, 281, 282, 283, 284, 285, 286, 287, 289, 290, 292, 296, 297, 298, 300, 302, 303, 310]
Malus pumila
[41]
Cornus mas
[41]
Prunus avium
[83, 150, 281, 282, 283, 284, 285, 286, 287, 289, 290, 292, 293, 296, 297, 298, 300, 302, 303, 334, 355, 358, 364, 365, 368, 373, 376, 377, 412, 413, 414, 417, 418, 419, 420, 422, 423, 424, 426, 427, 428, 429, 430, 432, 433, 435, 436, 438, 440, 441, 443, 444, 446, 447, 448, 449, 451, 452, 453, 454, 455, 456, 457, 551]
Prunus cerasus
[41, 177, 188, 284]
Prunus serotina
[41]
Pyrus communis
[83, 281, 289, 293, 296, 297, 298, 300, 302, 303, 334, 355, 358, 364, 365, 368, 373, 376, 377, 412, 413, 414, 417, 418, 419, 420, 422, 423, 424, 426, 427, 428, 429, 430, 432, 433, 435, 436, 438, 440, 441, 443, 444, 446, 447, 448, 449, 451, 452, 453, 454, 455, 456, 457]
Olea europea
[75, 83]
Citrus sinensis
[75]
Malus domestica
[83, 150, 281, 282, 283, 284, 28

In [ ]:
# Need to keep our monthly data as well. Rbind?

In [447]:
apple_train_df = universal_train_df[universal_train_df['formatted_sci_name'] == 'Malus domestica']

In [448]:
apple_model = train_ripeness(apple_train_df, euro_monthly,  # training data
                                                   test_dict['Malus domestica'], euro_monthly)

running model ThermalTime


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


making predictions for model ThermalTime
model ThermalTime got an aic of 31976.986553823604
Best model: ThermalTime
Best model paramters:
{'t1': 243.90771577137204, 'T': -17.330775186427342, 'F': 60.17335129293497}
Ripeness Day: 247.45335406486006


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


In [454]:
apple_model.keys()

dict_keys(['trained_model', 'model_aic', 'model_name', 'full_flowering_data', 'species_site_flowering days', 'mean_flowering_day'])

In [456]:
np.std(apple_model['species_site_flowering days'])

0.9793179289018654

In [457]:
np.mean(apple_model['species_site_flowering days'])

247.45335406486006

In [460]:
apple_model['full_flowering_data']

,doy,year,species,species_actual,phenophase,sci_name,latitude,longitude,site_id,formatted_sci_name,siteyear,flowering_day,model
3363,247.0,2013.0,Malus,domestica,516,Malus domestica,42,3,75,Malus domestica,752013,246,ThermalTime
3405,235.0,2013.0,Malus,domestica,516,Malus domestica,42,1,79,Malus domestica,792013,251,ThermalTime
3406,235.0,2013.0,Malus,domestica,516,Malus domestica,42,1,79,Malus domestica,792013,251,ThermalTime
3473,253.0,2011.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',45,15,83,Malus domestica,832011,246,ThermalTime
3474,270.0,2011.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',45,15,83,Malus domestica,832011,246,ThermalTime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27701,230.0,2013.0,Malus,domestica 'early cultivar',516,Malus domestica 'early cultivar',50,13,461,Malus domestica,4612013,249,ThermalTime
27702,244.0,2014.0,Malus,domestica 'early cultivar',516,Malus domestica 'early cultivar',50,13,461,Malus domestica,4612014,249,ThermalTime
27703,275.0,2011.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',50,13,461,Malus domestica,4612011,248,ThermalTime
27708,269.0,2012.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',50,13,461,Malus domestica,4612012,249,ThermalTime


In [463]:
apple_model['full_flowering_data'].groupby('year')['flowering_day'].mean()

year
2011.0    247.075805
2012.0    247.541401
2013.0    247.324895
2014.0    247.483412
2015.0    247.926526
2016.0    246.000000
Name: flowering_day, dtype: float64

In [466]:
apple_sites = apple_model['full_flowering_data']['site_id'].unique()

In [467]:
apple_sites

array([ 75,  79,  83, 150, 179, 281, 282, 283, 284, 285, 286, 287, 289,
       290, 292, 293, 294, 296, 297, 298, 300, 302, 303, 307, 313, 334,
       348, 355, 358, 364, 365, 368, 373, 376, 377, 412, 413, 414, 417,
       418, 419, 420, 422, 423, 424, 426, 427, 428, 429, 430, 432, 433,
       435, 436, 438, 440, 441, 443, 444, 446, 447, 448, 449, 451, 452,
       453, 454, 455, 456, 457, 461])

In [468]:
apple_weather = euro_monthly[euro_monthly['site_id'].isin(apple_sites)]

In [469]:
apple_weather.groupby('year')[['year', 'temperature']].mean()

,year,temperature
year,,
2011,2011.0,9.754072
2012,2012.0,9.358735
2013,2013.0,9.019795
2014,2014.0,10.479722
2015,2015.0,10.191269
2016,2016.0,9.666976
2017,2017.0,9.823329
2018,2018.0,10.597095
2019,2019.0,10.598027


In [475]:
np.std(apple_weather.groupby('site_id')[['site_id', 'temperature']].mean()['temperature'])

2.1427575779439594

In [476]:
np.mean(apple_weather.groupby('site_id')[['site_id', 'temperature']].mean()['temperature'])

9.959313284906296

In [477]:
euro_monthly.groupby('year')[['year', 'temperature']].mean()

,year,temperature
year,,
2011,2011.0,9.992545
2012,2012.0,9.604933
2013,2013.0,9.327666
2014,2014.0,10.676084
2015,2015.0,10.503460
2016,2016.0,10.003765
2017,2017.0,10.196167
2018,2018.0,10.842804
2019,2019.0,10.842869


In [478]:
site_based_models = {}

for species in tqdm(universal_train_df['formatted_sci_name'].unique()):
    print(species)
    
    species_train_df = universal_train_df[universal_train_df['formatted_sci_name'] == species]
    #print(species_df)
    species_test_df = test_dict[species]
    
    # no species filtering, the universal dataframe has plenty of data.
    
    site_based_models[species] = train_ripeness(species_train_df, euro_monthly,  # training data
                                                   species_test_df, euro_monthly) # test data 

  0%|                                                                                                                                                                              | 0/19 [00:00<?, ?it/s]/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


Ficus carica
running model ThermalTime


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
  5%|████████▋                                                                                                                                                             | 1/19 [00:07<02:13,  7.44s/it]

making predictions for model ThermalTime
model ThermalTime got an aic of 44.150290904222025
Best model: ThermalTime
Best model paramters:
{'t1': 226.59867980455525, 'T': 15.35309871142344, 'F': 673.5057940773045}
Ripeness Day: 511.3333333333333
Morus alba
running model ThermalTime


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
 11%|█████████████████▍                                                                                                                                                    | 2/19 [00:09<01:09,  4.07s/it]

...

making predictions for model ThermalTime
model ThermalTime got an aic of 55.42384020308592
Best model: ThermalTime
Best model paramters:
{'t1': 220.35282629344113, 'T': 22.875712919240137, 'F': 71.29089070703344}
Ripeness Day: 422.0
Citrus
running model ThermalTime


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎        | 18/19 [02:56<00:05,  5.98s/it]

making predictions for model ThermalTime
model ThermalTime got an aic of 28.379324478905936
Best model: ThermalTime
Best model paramters:
{'t1': 78.72544609110729, 'T': -13.066526444052517, 'F': 356.3373975954724}
Ripeness Day: 127.0
Rosmarinus officinalis
running model ThermalTime


/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [02:59<00:00,  9.43s/it]

making predictions for model ThermalTime
model ThermalTime got an aic of 21.83343004070722
Best model: ThermalTime
Best model paramters:
{'t1': 76.53225774883703, 'T': 5.655556781013393, 'F': 217.00570528675502}
Ripeness Day: 97.0


In [479]:
test_dataframes = []

for species in site_based_models:
    test_dataframes.append(site_based_models[species]['full_flowering_data'])
    
full_prediction_df = pd.concat(test_dataframes)

In [480]:
full_prediction_df

,doy,year,species,species_actual,phenophase,sci_name,latitude,longitude,site_id,formatted_sci_name,siteyear,flowering_day,model
1846,283.0,2015.0,Ficus,carica,516,Ficus carica,47,9,41,Ficus carica,412015,999,ThermalTime
2677,231.0,2017.0,Ficus,carica,516,Ficus carica,44,4,48,Ficus carica,482017,257,ThermalTime
2685,282.0,2018.0,Ficus,carica 'Brown Turkey',516,Ficus carica 'Brown Turkey',51,3,49,Ficus carica,492018,278,ThermalTime
2679,166.0,2017.0,Morus,alba,516,Morus alba,44,4,48,Morus alba,482017,185,ThermalTime
2680,166.0,2017.0,Morus,alba,516,Morus alba,44,4,48,Morus alba,482017,185,ThermalTime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27900,280.0,2017.0,Juglans,regia,516,Juglans regia,47,7,526,Juglans regia,5262017,241,ThermalTime
6585,346.0,2015.0,Citrus,x limon,516,Citrus x limon,42,13,236,Citrus,2362015,124,ThermalTime
6586,193.0,2019.0,Citrus,x aurantium,516,Citrus x aurantium,42,13,236,Citrus,2362019,130,ThermalTime
9831,78.0,2012.0,Rosmarinus,officinalis,516,Rosmarinus officinalis,45,14,310,Rosmarinus officinalis,3102012,91,ThermalTime


In [481]:
full_prediction_df['flowering_day'].unique()

array([999, 257, 278, 185, 198, 113, 105, 106, 235, 236, 239, 238, 237,
       242, 243, 240, 246, 241, 234, 247, 250, 251, 275, 248, 215,  87,
        60,  79,  58, 166, 167, 168, 170, 172, 169, 171, 176, 173, 177,
       174, 175, 165, 192, 200, 189, 187, 178, 182, 164, 186, 190, 188,
       180, 179, 191, 199, 193, 196, 184, 183, 154, 153, 253, 254, 264,
       256, 266, 255, 274, 292, 304, 290, 316, 252, 249, 145, 155, 152,
       302, 224, 124, 130,  91, 103])

In [484]:
full_prediction_df[full_prediction_df['flowering_day'] == 999]

,doy,year,species,species_actual,phenophase,sci_name,latitude,longitude,site_id,formatted_sci_name,siteyear,flowering_day,model
1846,283.0,2015.0,Ficus,carica,516,Ficus carica,47,9,41,Ficus carica,412015,999,ThermalTime
1856,237.0,2021.0,Prunus,domestica,516,Prunus domestica,47,9,41,Prunus domestica,412021,999,ThermalTime
1857,253.0,2021.0,Prunus,domestica,516,Prunus domestica,47,9,41,Prunus domestica,412021,999,ThermalTime
1902,182.0,2021.0,Prunus,cerasus,516,Prunus cerasus,47,9,41,Prunus cerasus,412021,999,ThermalTime
3401,294.0,2013.0,Olea,europea,516,Olea europea,42,1,79,Olea europea,792013,999,ThermalTime
3402,314.0,2013.0,Olea,europea,516,Olea europea,42,1,79,Olea europea,792013,999,ThermalTime
3403,285.0,2013.0,Olea,europea,516,Olea europea,42,1,79,Olea europea,792013,999,ThermalTime
3404,303.0,2013.0,Olea,europea,516,Olea europea,42,1,79,Olea europea,792013,999,ThermalTime
3436,288.0,2015.0,Olea,europea,516,Olea europea,45,15,83,Olea europea,832015,999,ThermalTime
3437,306.0,2015.0,Olea,europea,516,Olea europea,45,15,83,Olea europea,832015,999,ThermalTime


In [485]:
filtered_prediction_df = full_prediction_df[full_prediction_df['flowering_day'] != 999]

In [486]:
large_prediction_df = filtered_prediction_df[['species', 'formatted_sci_name', 'latitude', 'longitude', 'site_id', 'flowering_day']].drop_duplicates()


In [487]:
large_prediction_df

,species,formatted_sci_name,latitude,longitude,site_id,flowering_day
2677,Ficus,Ficus carica,44,4,48,257
2685,Ficus,Ficus carica,51,3,49,278
2679,Morus,Morus alba,44,4,48,185
4585,Morus,Morus alba,51,17,134,185
1849,Prunus,Prunus cerasifera,47,9,41,198
...,...,...,...,...,...,...
27900,Juglans,Juglans regia,47,7,526,241
6585,Citrus,Citrus,42,13,236,124
6586,Citrus,Citrus,42,13,236,130
9831,Rosmarinus,Rosmarinus officinalis,45,14,310,91


Why is it over 365?

In [488]:
large_prediction_df.to_csv("../data/model_training_data/daily_european_site_specific_ripeness_sci_names.csv")

In [ ]:
large_prediction_df['formatted_ripeness_date'] = pd.to_datetime(model_output_df['mean_ripeness'], format='%j').dt.strftime('%m-%d')



In [385]:
def get_ripeness_days(model_output):
    ripeness_days = {}
    
    for k in model_output:
        ripeness_days[k] = {'mean_ripeness': model_output[k]['mean_flowering_day'],
                            'sd_ripeness':  np.std(model_output[k]['species_site_flowering days'])}
    
    return ripeness_days

In [387]:
model_output_df = pd.DataFrame.from_dict(get_ripeness_days(site_based_models), "index").reset_index().rename({'index': 'species'},axis=1)
model_output_df['formatted_ripeness_date'] = pd.to_datetime(model_output_df['mean_ripeness'], format='%j').dt.strftime('%m-%d')

model_output_df['mean_ripeness'] = np.around(model_output_df['mean_ripeness'], 1)
model_output_df['sd_ripeness'] = np.around(model_output_df['sd_ripeness'], 1)

In [388]:
model_output_df

,species,mean_ripeness,sd_ripeness,formatted_ripeness_date
0,Ficus carica,266.7,0.5,09-23
1,Morus alba,172.5,0.5,06-21
2,Prunus cerasifera,193.0,0.0,07-12
3,Prunus domestica,237.4,0.5,08-25
4,Malus pumila,258.2,0.4,09-15
5,Cornus mas,249.5,0.5,09-06
6,Prunus avium,174.4,0.5,06-23
7,Prunus cerasus,176.2,0.4,06-25
8,Prunus serotina,187.0,0.0,07-06
9,Pyrus communis,252.5,0.5,09-09


## Test Overfitting

In [395]:
dict(universal_train_df.groupby('site_id').size())

{41: 15,
 48: 2,
 49: 1,
 75: 2,
 83: 36,
 128: 4,
 150: 22,
 162: 1,
 177: 1,
 186: 2,
 188: 7,
 261: 1,
 281: 98,
 282: 27,
 283: 72,
 284: 54,
 285: 15,
 286: 21,
 287: 36,
 289: 27,
 290: 28,
 292: 5,
 293: 92,
 296: 38,
 297: 54,
 298: 65,
 300: 38,
 302: 42,
 303: 32,
 310: 8,
 313: 3,
 334: 64,
 355: 42,
 358: 95,
 361: 1,
 364: 189,
 365: 56,
 368: 150,
 373: 115,
 376: 86,
 377: 129,
 412: 150,
 413: 212,
 414: 188,
 417: 120,
 418: 53,
 419: 119,
 420: 65,
 422: 192,
 423: 161,
 424: 141,
 426: 81,
 427: 204,
 428: 86,
 429: 178,
 430: 14,
 432: 67,
 433: 186,
 435: 151,
 436: 132,
 438: 93,
 440: 68,
 441: 119,
 443: 50,
 444: 144,
 446: 73,
 447: 82,
 448: 111,
 449: 102,
 451: 34,
 452: 24,
 453: 62,
 454: 23,
 455: 39,
 456: 210,
 457: 30,
 461: 4,
 551: 8}

In [408]:

big_site = universal_train_df[(universal_train_df['site_id'] == 413) & (universal_train_df['formatted_sci_name'] == 'Malus domestica')]

big_site_test = test_dict['Malus domestica']

big_site_test = big_site_test[big_site_test['site_id'] == 413]


In [409]:
big_site_test

,doy,year,species,species_actual,phenophase,sci_name,latitude,longitude,site_id,formatted_sci_name,siteyear,flowering_day,model
13852,204.0,2011.0,Malus,domestica 'early cultivar',516,Malus domestica 'early cultivar',49,9,413,Malus domestica,4132011,247,ThermalTime
13853,208.0,2012.0,Malus,domestica 'early cultivar',516,Malus domestica 'early cultivar',49,9,413,Malus domestica,4132012,248,ThermalTime
13854,213.0,2013.0,Malus,domestica 'early cultivar',516,Malus domestica 'early cultivar',49,9,413,Malus domestica,4132013,248,ThermalTime
13856,222.0,2015.0,Malus,domestica 'early cultivar',516,Malus domestica 'early cultivar',49,9,413,Malus domestica,4132015,248,ThermalTime
13858,228.0,2011.0,Malus,domestica 'early cultivar',516,Malus domestica 'early cultivar',49,9,413,Malus domestica,4132011,247,ThermalTime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14214,291.0,2013.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',49,9,413,Malus domestica,4132013,248,ThermalTime
14215,278.0,2014.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',49,9,413,Malus domestica,4132014,248,ThermalTime
14216,263.0,2015.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',49,9,413,Malus domestica,4132015,248,ThermalTime
14218,303.0,2014.0,Malus,domestica 'late cultivar',516,Malus domestica 'late cultivar',49,9,413,Malus domestica,4132014,248,ThermalTime


In [416]:
len(list(big_site_test['doy']))

198

In [420]:
np.mean(big_site_test['doy'])

243.8838383838384

In [418]:
apple_trained_model = site_based_models['Malus domestica']['trained_model']

dict(zip(list(big_site_test['doy']), apple_trained_model.predict(big_site_test, euro_monthly)))

/Users/peterbenson/opt/anaconda3/envs/seasonality/lib/python3.11/site-packages/pyPhenology/models/utils/misc.py:66: UserWarning: Dropped temperature data for doy 366 due to missing data. Most likely from leap year mismatch
  warn("""Dropped temperature data for doy {d} due to missing data. Most likely from leap year mismatch""".format(d=last_doy_column))


{204.0: 248,
 208.0: 248,
 213.0: 248,
 222.0: 248,
 228.0: 248,
 236.0: 248,
 193.0: 248,
 202.0: 248,
 227.0: 248,
 219.0: 247,
 220.0: 248,
 240.0: 248,
 218.0: 248,
 187.0: 247,
 225.0: 247,
 191.0: 248,
 188.0: 248,
 200.0: 247,
 209.0: 247,
 201.0: 248,
 205.0: 248,
 203.0: 248,
 183.0: 247,
 230.0: 248,
 229.0: 248,
 206.0: 248,
 199.0: 247,
 214.0: 248,
 221.0: 248,
 197.0: 247,
 217.0: 248,
 243.0: 248,
 235.0: 248,
 184.0: 247,
 207.0: 248,
 186.0: 248,
 196.0: 248,
 226.0: 247,
 189.0: 248,
 192.0: 247,
 212.0: 247,
 195.0: 248,
 190.0: 248,
 223.0: 248,
 262.0: 247,
 275.0: 248,
 276.0: 247,
 259.0: 248,
 269.0: 247,
 257.0: 248,
 256.0: 248,
 265.0: 248,
 270.0: 248,
 274.0: 248,
 271.0: 248,
 263.0: 248,
 294.0: 248,
 282.0: 248,
 283.0: 248,
 290.0: 248,
 268.0: 247,
 261.0: 248,
 277.0: 248,
 291.0: 248,
 289.0: 248,
 293.0: 248,
 267.0: 248,
 278.0: 248,
 266.0: 247,
 292.0: 248,
 281.0: 248,
 285.0: 248,
 299.0: 247,
 307.0: 248,
 273.0: 248,
 248.0: 248,
 237.0: 248,

Problem is with prediction, not with training. The training is definitely doing ok, it's "accurate" but the prediction isn't "precise". 

TODO: more experimentation with predictions

## Predictor Refinement

In [421]:
euro_monthly.groupby('year')[['year', 'temperature']].mean()


,year,temperature
year,,
2011,2011.0,283.142545
2012,2012.0,282.754933
2013,2013.0,282.477666
2014,2014.0,283.826084
2015,2015.0,283.653460
2016,2016.0,283.153765
2017,2017.0,283.346167
2018,2018.0,283.992804
2019,2019.0,283.992869


In [493]:
np.sqrt(np.mean(np.abs(filtered_prediction_df['flowering_day'] - filtered_prediction_df['doy'])))

4.855158241811518

In [ ]:
for s in filtered_prediction_df['formatted_sci_name'].unique():
    species_df = filtered_prediction_df[filtered_prediction_df['']]